Coastal change is a global phenomenon that is attributed to tides, powerful sea currents and overall climate change. It has the potential to threaten communities and local economies of coastal towns and cities.

We developed an end-to-end GBDX workflow for coastal change detection and measurement at the native resolution of our 8-band multispectral imagery. This example focuses on Cape Cod, an area which is well known for extreme changes in the coastal landscape. The workflow takes two roughly collocated images of Cape Cod, captured in 2010 and 2016 by WorldView-2 and WorldView-3, respectively, and computes coastal change on the entire images, roughly an area of 1500km2 in less than 30 minutes.

The full story is [here](http://gbdxstories.digitalglobe.com/coastal-change/).

In [2]:
# Specify your credentials and create a gbdx interface

import os
os.environ['GBDX_USERNAME'] = ''
os.environ['GBDX_PASSWORD'] = ''
os.environ['GBDX_CLIENT_ID'] = '' 
os.environ['GBDX_CLIENT_SECRET'] = ''

import gbdxtools
gbdx = gbdxtools.Interface()

In [2]:
import uuid
from os.path import join

# S3 location of the two images
input_location = 's3://gbd-customer-data/32cbab7a-4307-40c8-bb31-e2de32f940c2/platform-stories/coastal-change/images'

# Image alignment
ipa = gbdx.Task('protogenV2CD_READY')
ipa.inputs.raster = join(input_location, 'post')
ipa.inputs.slave  = join(input_location, 'pre')

# Water masks
water_pre = gbdx.Task('protogenV2RAW')
water_post = gbdx.Task('protogenV2RAW')
water_pre.inputs.raster = ipa.outputs.slave.value
water_post.inputs.raster = ipa.outputs.data.value

# Exclusion mask
exclusion_mask = gbdx.Task('protogenV2CD_LULC')
exclusion_mask.domain = 'raid'
exclusion_mask.inputs.raster = ipa.outputs.data.value
exclusion_mask.inputs.slave = ipa.outputs.slave.value

# Water gain map
bcd_gain = gbdx.Task('protogenV2CD_BIN_GAIN')
bcd_gain.inputs.raster = water_post.outputs.data.value
bcd_gain.inputs.slave  = water_pre.outputs.data.value
bcd_gain.inputs.mask   = exclusion_mask.outputs.data.value

# Water loss map
bcd_loss = gbdx.Task('protogenV2CD_BIN_LOSS')
bcd_loss.inputs.raster = water_post.outputs.data.value
bcd_loss.inputs.slave  = water_pre.outputs.data.value
bcd_loss.inputs.mask   = exclusion_mask.outputs.data.value

# Tristate map
bcd_tri = gbdx.Task('protogenV2CD_BIN_TRI')
bcd_tri.inputs.raster = water_post.outputs.data.value
bcd_tri.inputs.slave  = water_pre.outputs.data.value
bcd_tri.inputs.mask   = exclusion_mask.outputs.data.value

# Discrete distance transform of gain and loss maps
ddt_gain = gbdx.Task('protogenV2CD_GDDT')
ddt_gain.inputs.raster = water_post.outputs.data.value
ddt_gain.inputs.slave = bcd_gain.outputs.data.value
ddt_loss = gbdx.Task('protogenV2CD_LDDT')
ddt_loss.inputs.raster = water_post.outputs.data.value
ddt_loss.inputs.slave  = bcd_loss.outputs.data.value

# Define the workflow
wf = gbdx.Workflow([ipa, water_pre, water_post, exclusion_mask, bcd_tri, bcd_loss, bcd_gain, ddt_gain, ddt_loss])

# Define output location
random_str = str(uuid.uuid4())
output_location = join('platform-stories/trial-runs', random_str)

# Define where to save outputs from various tasks
wf.savedata(water_pre.outputs.data, output_location + '/water_pre')
wf.savedata(water_post.outputs.data, output_location + '/water_post')
wf.savedata(exclusion_mask.outputs.data, output_location + '/exclusion_mask')
wf.savedata(bcd_tri.outputs.data, output_location + '/bcd_tristate')
wf.savedata(bcd_gain.outputs.data, output_location + '/bcd_gain')
wf.savedata(bcd_loss.outputs.data, output_location + '/bcd_loss')
wf.savedata(ddt_gain.outputs.data, output_location + '/ddt_gain')
wf.savedata(ddt_loss.outputs.data, output_location + '/ddt_loss')

In [ ]:
# Execute the workflow
wf.execute()

In [ ]:
wf.status

In [ ]:
# Check under this location for the output
print output_location

In [3]:
# Create slippy map
from ipyleaflet import Map, TileLayer

m = Map(center=[41.67, -70.00], zoom=12)

# These are the IDAHO TMS urls for the pre image and for the tristate map
# We've saved them in IDAHO format in a custom bucket and are serving them through the TMS service
url_pre = 'http://idaho.geobigdata.io/v1/tile/platform-stories/0a3e570e-2545-4dae-ae89-37d78f8370c7/{z}/{x}/{y}?bands=4,2,1&token=' + gbdx.gbdx_connection.access_token
url_tristate = 'http://idaho.geobigdata.io/v1/tile/platform-stories/c1a57115-af5c-4286-b93c-59009c7ff6c2/{z}/{x}/{y}?bands=0,1,2&token=' + gbdx.gbdx_connection.access_token

m.add_layer(TileLayer(url=url_pre))
m.add_layer(TileLayer(url=url_tristate, opacity=0.5))
    
# launch map    
m